# ResNet50 Multi-label Xray Model 
1.  Architecture:
- Uses ResNet50 (ImageNet-pretrained) as the feature extractor with a custom head:
-  Global Average Pooling → Dense(256, ReLU) → Dropout(0.4) → Dense(14, Sigmoid).
2. Data Loading:
- Uses a custom XRayDataGenerator (OpenCV-based) for efficient image loading, resizing, and preprocessing.
3. Optimization:
- Mixed precision (float16) + XLA compilation for faster GPU performance.
- CosineDecayRestarts learning-rate schedule.
- Adam optimizer with binary cross-entropy loss.
4. Training Strategy:
- 2-fold Multilabel Stratified K-Fold CV for robust validation.
5. Two-stage training:
- Train top layers (frozen base).
- Fine-tune last 20 ResNet layers.
6. Callbacks:
- ModelCheckpoint (saves best model by val_AUC) and EarlyStopping (patience=5).
7. Output:
- Saves each fold’s best/final models.
- Creates an ensemble of 5 folds (average predictions).
Exports final submission.csv for Kaggle.

In [2]:
import tensorflow as tf
print("GPUs Available:", len(tf.config.list_physical_devices('GPU')))


GPUs Available: 1


In [17]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import cv2
import os
import sys
import os, gc
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras import mixed_precision
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from io import StringIO
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import tensorflow.keras.applications.resnet50 as resnet
import warnings
from tensorflow.keras.utils import Sequence
import tensorflow.keras.applications.resnet50 as resnet
warnings.filterwarnings('ignore')

In [10]:
!pip install iterative-stratification

In [4]:
print(os.listdir("/kaggle/input"))

# Path to competition dataset
data_dir = "/kaggle/input/grand-xray-slam-division-b"
# Check what files are inside
print('Filenames of the data', os.listdir(data_dir))

['grand-xray-slam-division-b']
Filenames of the data ['test2', 'sample_submission_2.csv', 'train2.csv', 'train2']


In [5]:
# Load the training CSV metadata with labels
train = pd.read_csv("/kaggle/input/grand-xray-slam-division-b/train2.csv")

print('Metadata shape:',train.shape)
train.head()

Metadata shape: (108494, 21)


,Image_name,Patient_ID,Study,Sex,Age,ViewCategory,ViewPosition,Atelectasis,Cardiomegaly,Consolidation,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,00000003_001_001.jpg,3,1,Male,41.0,Frontal,AP,0,1,0,...,1,0,0,1,0,0,0,0,0,0
1,00000004_001_001.jpg,4,1,Female,20.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,00000004_001_002.jpg,4,1,Female,20.0,Lateral,Lateral,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,00000006_001_001.jpg,6,1,Female,42.0,Frontal,AP,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,00000010_001_001.jpg,10,1,Female,50.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [6]:
# 1. Feature & Target Preperation
# Define labels
conditions = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
    'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
    'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'
]
# Features you want
features = ["ViewCategory", "ViewPosition", "Age", "Sex"]

# Encode categorical features
from sklearn.preprocessing import LabelEncoder

train_enc = train.copy()   # train data encoded
for col in ["ViewCategory", "ViewPosition", "Sex"]:  # features that can be encoded
    le = LabelEncoder()
    train_enc[col] = le.fit_transform(train[col].astype(str))

X = train_enc[features].values
y = train[conditions].values
print(X.shape) # 4 features (ViewCategory, ViewPosition, Age, Sex)
print(y.shape)  # 14 conditions

(108494, 4)
(108494, 14)


In [7]:
# 2. Adding ViewBalancing for Stratification: ViewCategory= Frontal, Lateral; since ViewCategory is unbalanced

# One-hot encode ViewCategory and append to 
view_onehot = pd.get_dummies(train["ViewCategory"], prefix="view").values

y_aug = np.hstack([y, view_onehot])  # augmented target matrix (added ViewCategory as y to stratify and reduce bias)

# Data Generator

In [18]:
# Data Generator (OpenCV based)
class XRayDataGenerator(Sequence):
    def __init__(self, dataframe, batch_size=32, img_size=(224, 224), is_test=False, **kwargs):
        super().__init__(**kwargs)
        self.dataframe = dataframe.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size = img_size
        self.is_test = is_test
        self.image_dir = '/kaggle/input/grand-xray-slam-division-b/train2/' if not is_test else '/kaggle/input/grand-xray-slam-division-b/test2/'
        self.conditions = conditions
        
        if not os.path.exists(self.image_dir):
            raise FileNotFoundError(f"Directory {self.image_dir} not found.")
    
    def __len__(self):
        return (len(self.dataframe) + self.batch_size - 1) // self.batch_size
    
    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min(start + self.batch_size, len(self.dataframe))
        batch_data = self.dataframe.iloc[start:end]
        
        images, labels = [], []
        for _, row in batch_data.iterrows():
            # image loading
            img_path = os.path.join(self.image_dir, row['Image_name'])
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            
            if img is not None:
                # image augmentation: resize and preprocess using resnet
                img = cv2.resize(img, self.img_size)
                img = resnet.preprocess_input(img)
                images.append(img)
                if not self.is_test:
                    labels.append(row[self.conditions].values.astype(np.float32))
        
        if not images:
            images.append(np.zeros((*self.img_size, 3), dtype=np.float32))
            if not self.is_test:
                labels.append(np.zeros(len(self.conditions), dtype=np.float32))
        
        return (np.array(images), np.array(labels)) if not self.is_test else np.array(images)


# ResNet 
**2-Fold stratified training with mixed precision, cosine LR schedule, and fine-tuning of top ResNet layers.
Uses custom OpenCV generator + ensemble of folds for strong, GPU-optimized AUC performance.**

1. fold 1: 3 epochs frozen, 3 epochs trainable 20 layers saved as **fold_0_final.h5**
2. fold 2: 3 epochs frozen, 3 epochs trainable 20 layers

In [20]:
# Enable GPU optimizations
tf.config.optimizer.set_jit(True)
tf.config.experimental.enable_tensor_float_32_execution(True)
mixed_precision.set_global_policy('mixed_float16')
print("✅ Mixed precision + XLA enabled for faster GPU performance.")

print('**********Building ResNet Model******************')
#  Build ResNet50 model
# ============================================================
def build_resnet_model(num_classes=14, unfreeze_layers=None):
    base = ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))
    if unfreeze_layers:
        for layer in base.layers[-unfreeze_layers:]:
            layer.trainable = True
    else:
        base.trainable = False

    x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    out = tf.keras.layers.Dense(num_classes, activation="sigmoid")(x)
    return tf.keras.Model(inputs=base.input, outputs=out)

# ============================================================
# ⚙️ Callbacks
# ============================================================
def get_callbacks(fold):
    return [
        ModelCheckpoint(f"fold_{fold}_best.h5", monitor="val_auc", mode="max", save_best_only=True, verbose=1),
        EarlyStopping(monitor="val_auc", mode="max", patience=5, restore_best_weights=True, verbose=1),
    ]

# ============================================================
#  Cross-validation training loop
# ============================================================
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# Try to load existing AUCs if you already ran some folds
try:
    fold_aucs
except NameError:
    fold_aucs = []
print('**********Starting MSKF (k-fold) and CV training******************')
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
# *********do two folds only************************** if fold >1: break
for fold, (train_idx, val_idx) in enumerate(mskf.split(X, y_aug)):
    # skip already completed folds
    if os.path.exists(f"fold_{fold}_final.h5"):
        print(f"✅ Skipping fold {fold} (already completed)")
        continue
    if fold > 1:  # stop after fold 2
        break
    print(f"\n================ FOLD {fold+1} ================")
    train_df = train.iloc[train_idx].reset_index(drop=True)
    val_df   = train.iloc[val_idx].reset_index(drop=True)

    # Use generators
    train_generator = XRayDataGenerator(train_df, batch_size=BATCH_SIZE, img_size=IMG_SIZE)
    val_generator   = XRayDataGenerator(val_df, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

    # Learning rate schedule
    lr_schedule = CosineDecayRestarts(
        initial_learning_rate=1e-4,
        first_decay_steps=len(train_generator)*2,
        t_mul=2.0, m_mul=0.9, alpha=1e-6
    )
    # building resnet architecture 
    model = build_resnet_model() 
    model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc")]
    )
    # decrease epochs for less time
    history = model.fit(
        train_generator,validation_data=val_generator,
        epochs=3, callbacks=get_callbacks(fold), verbose=1)

    # Fine-tuning last 20 layers
    for layer in model.layers[-20:]:
        layer.trainable = True
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5), loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc")]
    )
    # decrease epochs for less time
    history_ft = model.fit(
        train_generator, validation_data=val_generator,
        epochs=3, callbacks=get_callbacks(fold), verbose=1)

    model.save(f"fold_{fold}_final.h5")
    key = 'val_auc' if 'val_auc' in history.history else 'val_AUC'
    best_auc = max(history.history[key] + history_ft.history[key])
    fold_aucs.append(best_auc)
    print(f"✅ Fold {fold+1} Best AUC: {best_auc:.4f}")
    gc.collect()

print('*************************MSKF K-FOld COMPLETE*************')

# ============================================================
# 🧾 CV Summary
# ============================================================
print(f"\n📊 Cross-validation AUCs: {fold_aucs}")
print(f"🏆 Mean CV AUC: {np.mean(fold_aucs):.4f}")

✅ Mixed precision + XLA enabled for faster GPU performance.
**********Building ResNet Model******************
**********Starting MSKF (k-fold) and CV training******************

================ FOLD 1 ================
Epoch 1/3
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 5030s 2s/step - auc: 0.8180 - loss: 0.4265 - val_auc: 0.8922 - val_loss: 0.3453
Epoch 2/3
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 5067s 2s/step - auc: 0.8814 - loss: 0.3543 - val_auc: 0.8957 - val_loss: 0.3400
Epoch 3/3
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 4941s 2s/step - auc: 0.8842 - loss: 0.3575 - val_auc: 0.9007 - val_loss: 0.3325
Epoch 1/3
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 4921s 2s/step - auc: 0.8899 - loss: 0.3441 - val_auc: 0.9008 - val_loss: 0.3381
Epoch 2/3
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 4971s 2s/step - auc: 0.9081 - loss: 0.3196 - val_auc: 0.8955 - val_loss: 0.3434
Epoch 3/3
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 4953s 2s/step - auc: 0.9148 - loss: 0.3085 - val_auc: 0.8936 - val_loss: 0.3467


KeyError: 'val_AUC'

In [40]:
best_auc = max(history.history['val_auc'] + history_ft.history['val_auc'])
# fold_aucs.append(best_auc)
print(f"✅ Fold {fold+1} Best AUC: {best_auc:.4f}")

# ============================================================
# 🧾 CV Summary
# ============================================================
print(f"\n📊 Cross-validation AUCs: {fold_aucs}")
print(f"🏆 Mean CV AUC: {np.mean(fold_aucs):.4f}")

✅ Fold 1 Best AUC: 0.9008

📊 Cross-validation AUCs: [0.9008244276046753]
🏆 Mean CV AUC: 0.9008


In [37]:
print(history.history)   # 3 epochs frozen
history_ft.history       # 3 epochs trained layers

{'auc': [0.856939971446991, 0.8844783306121826, 0.8878996968269348], 'loss': [0.39033958315849304, 0.3552301526069641, 0.35056841373443604], 'val_auc': [0.89216548204422, 0.8957088589668274, 0.9007192254066467], 'val_loss': [0.34534314274787903, 0.3400052487850189, 0.33251953125]}


{'auc': [0.8944286108016968, 0.9066745042800903, 0.9145926833152771],
 'loss': [0.3409472703933716, 0.32190805673599243, 0.30880698561668396],
 'val_auc': [0.9008244276046753, 0.8954897522926331, 0.8935645818710327],
 'val_loss': [0.3380545377731323, 0.343383252620697, 0.3467102348804474]}

In [41]:
# # Inferences
# test_df = pd.read_csv("/kaggle/input/grand-xray-slam-division-b/sample_submission_2.csv")
# test_df["Image_name"] = test_df["Image_name"].astype(str)
# test_generator = XRayDataGenerator(test_df, batch_size=BATCH_SIZE, img_size=IMG_SIZE, is_test=True)

# fold_preds = []
# for fold in range(2):

#     m = tf.keras.models.load_model(f"fold_{fold}_best.h5", compile=False)
#     fold_preds.append(m.predict(test_generator, verbose=1))

# final_preds = np.mean(fold_preds, axis=0)
# submission = pd.DataFrame(final_preds, columns=conditions)
# submission.insert(0, "Image_name", test_df["Image_name"].values)
# submission.to_csv("submission.csv", index=False)
# print("✅ submission.csv created successfully!")
